<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/regrid_land_parcels_maps_spatial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regrid | Land Parcels, Maps, and Spatial Analysis

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 1st July 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Regrid Map Search](https://app.regrid.com/us/ca/fresno)
- [Regrid API](https://regrid.com/api )
- [Regrid Plans](https://regrid.com/api/plans )
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

In [ ]:
# any outside packages install here
!pip install keplergl -q

     |████████████████████████████████| 9.7 MB 8.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 49.2 MB/s 
     |████████████████████████████████| 16.7 MB 36.0 MB/s 
     |████████████████████████████████| 6.3 MB 52.2 MB/s 


## <font color="blue">Imports</font>

In [ ]:
from google.colab import output, drive, files # google colab specific
import os
import warnings
import requests
import pandas as pd
import plotly.express as px
from keplergl import KeplerGl

pd.set_option('display.max_columns', None)
output.enable_custom_widget_manager()

## <font color="blue">Functions</font>

In [ ]:
def download_csv(df, filename="output"):
  # download file
  output_filename = '{}.csv'.format(filename)
  df.to_csv(output_filename, index=False)
  files.download(output_filename)

In [ ]:
def transform_kepler(df, polygon_geometry_col='geometry.coordinates'):
  data_dict = {}
  features_list = []
  relevant_cols = [c for c in df.columns if 'properties.fields' in c]
  for index, row in df.iterrows():
    d = {}
    d['type'] = 'Feature'
    d['geometry'] = {}
    d['geometry']['coordinates'] = row[polygon_geometry_col]
    d['geometry']['type'] = 'Polygon'
    d['properties'] = {}
    # get relevant columns
    for c in relevant_cols:
      d['properties'][c] = row[c]
    features_list.append(d)

  data_dict = {
      'type': 'FeatureCollection',
      'features': features_list
  }

  return data_dict

## <font color="blue">Locals & Constants</font>

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set working directory
os.chdir('/content/drive/MyDrive/Colab Data/')
dir = os.getcwd()
print('Current working directory:', dir)

# remove pandas warnings
warnings.filterwarnings('ignore')

Current working directory: /content/drive/MyDrive/Colab Data


## <font color="blue">Data</font>

In [ ]:
# read in api keys
df_api_keys = pd.read_csv(dir + '/input/api_keys.csv')
# extract api key
regrid_api_key = df_api_keys.loc[df_api_keys['API'] == 'regrid']['KEY'].iloc[0]
mapbox_api_key = df_api_keys.loc[df_api_keys['API'] == 'mapbox']['KEY'].iloc[0]

## <font color="blue">API: Parcel API search</font>
<b>Description:</b> Search parcels (land) within a specific area or address

<b>Search Criteria:</b>

1. LAT/LON

2. ADDRESS

3. PARCEL ID

4. POLYGON

5. OWNER NAME

6. LL_UUID (UNIVERSAL UNIQUE ID)

### <font color="purple">Use Case #1: Lat/Lon (Reverse Geocoding) Search</font>

In [ ]:
# end point
url = "https://app.regrid.com/api/v1/search.json"

# parameters
querystring = {
    "lat":"27.967220",
    "lon":"-82.438350",
    "nearest":"1",
    "radius":"250",
    "limit":50, # default is 20
    "token":regrid_api_key
}

response = requests.request("GET", url, params=querystring)

print(response.text)

{"results":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-82.4383479519374,27.9671946152829],[-82.4383492375205,27.9673134909687],[-82.4383508866368,27.9674682326501],[-82.4384551624888,27.967467515631],[-82.4384512998687,27.9671944787614],[-82.4383479519374,27.9671946152829]]]},"properties":{"headline":"1805 E 17th Ave","path":"/us/fl/hillsborough/tampa/469030","fields":{"ogc_fid":469030,"geoid":"12057","parcelnumb":"187615.0000","parcelnumb_no_formatting":"1876150000","alt_parcelnumb1":"1876150000","alt_parcelnumb2":"A-07-29-19-4VF-000006-00003.0","usecode":"0100","usedesc":"SINGLE FAMILY","structno":1,"yearbuilt":1920,"numstories":1.0,"numunits":1,"parvaltype":"MARKET","improvval":48327.0,"landval":38220.0,"parval":87217.0,"saleprice":0.0,"owner":"HUMBERTO ENRIQUEZ","mailadd":"2707 OWEN ST","mail_city":"TAMPA","mail_state2":"FL","mail_zip":"33605-2654","address":"1805 E 17TH AVE","ll_address_count":1,"ll_bldg_footprint_sqft":554,"ll_bldg_count":1,"dpv_status":"V",

In [ ]:
print('HTTP Status Code:', response.status_code)
response_json = response.json()

HTTP Status Code: 200


In [ ]:
response.json().keys()

dict_keys(['results'])

In [ ]:
df = pd.json_normalize(response_json, record_path=['results'])
print('Num of rows:', len(df))
print('Num of columns:', len(df.columns))
df.head()

Num of rows: 50
Num of columns: 255


,type,id,geometry.type,geometry.coordinates,properties.headline,properties.path,properties.fields.ogc_fid,properties.fields.geoid,properties.fields.parcelnumb,properties.fields.parcelnumb_no_formatting,properties.fields.alt_parcelnumb1,properties.fields.alt_parcelnumb2,properties.fields.usecode,properties.fields.usedesc,properties.fields.structno,properties.fields.yearbuilt,properties.fields.numstories,properties.fields.numunits,properties.fields.parvaltype,properties.fields.improvval,properties.fields.landval,properties.fields.parval,properties.fields.saleprice,properties.fields.owner,properties.fields.mailadd,properties.fields.mail_city,properties.fields.mail_state2,properties.fields.mail_zip,properties.fields.address,properties.fields.ll_address_count,properties.fields.ll_bldg_footprint_sqft,properties.fields.ll_bldg_count,properties.fields.dpv_status,properties.fields.dpv_codes,properties.fields.dpv_notes,properties.fields.dpv_type,properties.fields.cass_errorno,properties.fields.rdi,properties.fields.usps_vacancy,properties.fields.usps_vacancy_date,properties.fields.lbcs_activity,properties.fields.lbcs_activity_desc,properties.fields.lbcs_function,properties.fields.lbcs_function_desc,properties.fields.lbcs_structure,properties.fields.lbcs_structure_desc,properties.fields.lbcs_site,properties.fields.lbcs_site_desc,properties.fields.saddno,properties.fields.saddpref,properties.fields.saddstr,properties.fields.saddsttyp,properties.fields.scity,properties.fields.original_address,properties.fields.city,properties.fields.county,properties.fields.state2,properties.fields.szip,properties.fields.address_source,properties.fields.legaldesc,properties.fields.lat,properties.fields.lon,properties.fields.qoz,properties.fields.qoz_tract,properties.fields.census_tract,properties.fields.census_block,properties.fields.census_blockgroup,properties.fields.ll_last_refresh,properties.fields.recrdareano,properties.fields.gisacre,properties.fields.ll_gisacre,properties.fields.ll_gissqft,properties.fields.plss_township,properties.fields.plss_section,properties.fields.plss_range,properties.fields.path,properties.fields.ll_stable_id,properties.fields.ll_uuid,properties.fields.ll_updated_at,properties.fields.asd_val,properties.fields.sub,properties.fields.strap,properties.fields.taxdist,properties.fields.exf,properties.fields.eff,properties.fields.tax_val,properties.fields.tbeds,properties.fields.tbaths,properties.fields.muni,properties.fields.tif,properties.fields.nbhc,properties.fields.base,properties.fields.just,properties.fields.folio,properties.fields.pin,properties.field_labels.ogc_fid,properties.field_labels.geoid,properties.field_labels.sourceagent,properties.field_labels.parcelnumb,properties.field_labels.parcelnumb_no_formatting,properties.field_labels.alt_parcelnumb1,properties.field_labels.alt_parcelnumb2,properties.field_labels.alt_parcelnumb3,properties.field_labels.usecode,properties.field_labels.usedesc,properties.field_labels.zoning,properties.field_labels.zoning_description,properties.field_labels.struct,properties.field_labels.multistruct,properties.field_labels.structno,properties.field_labels.yearbuilt,properties.field_labels.numstories,properties.field_labels.numunits,properties.field_labels.structstyle,properties.field_labels.parvaltype,properties.field_labels.improvval,properties.field_labels.landval,properties.field_labels.parval,properties.field_labels.agval,properties.field_labels.saleprice,properties.field_labels.saledate,properties.field_labels.taxamt,properties.field_labels.taxyear,properties.field_labels.owntype,properties.field_labels.owner,properties.field_labels.ownfrst,properties.field_labels.ownlast,properties.field_labels.owner2,properties.field_labels.owner3,properties.field_labels.owner4,properties.field_labels.subsurfown,properties.field_labels.subowntype,properties.field_labels.mailadd,properties.field_labels.mail_address2,properties.field_labels.careof,properties.field_labels.mail_addno,properties.field_label

In [ ]:
df.groupby(['properties.fields.usedesc'])['id']\
  .count().reset_index().rename(columns={'id': 'count'})\
  .sort_values(by=['count'], ascending=False)

,properties.fields.usedesc,count
2,SINGLE FAMILY,35
3,VACANT RESIDENT,11
0,GOVERNMENT,2
1,MULTI-FAMILY,2


In [ ]:
# get list of unique column names
col_list = list(set(c.split('.')[-1] for c in df.columns))
col_list.sort()
col_list

['active',
 'address',
 'address2',
 'address_source',
 'agval',
 'alt_parcelnumb1',
 'alt_parcelnumb2',
 'alt_parcelnumb3',
 'asd_val',
 'base',
 'block',
 'book',
 'careof',
 'cass_errorno',
 'cdl_date',
 'cdl_majority_category',
 'cdl_majority_percent',
 'cdl_raw',
 'census_block',
 'census_blockgroup',
 'census_tract',
 'city',
 'coordinates',
 'county',
 'dba',
 'dpv_codes',
 'dpv_notes',
 'dpv_status',
 'dpv_type',
 'eff',
 'exf',
 'folio',
 'geoid',
 'gisacre',
 'headline',
 'homestead_exemption',
 'id',
 'improvval',
 'just',
 'landval',
 'lat',
 'lbcs_activity',
 'lbcs_activity_desc',
 'lbcs_function',
 'lbcs_function_desc',
 'lbcs_ownership',
 'lbcs_ownership_desc',
 'lbcs_site',
 'lbcs_site_desc',
 'lbcs_structure',
 'lbcs_structure_desc',
 'legaldesc',
 'll_address_count',
 'll_bldg_count',
 'll_bldg_footprint_sqft',
 'll_gisacre',
 'll_gissqft',
 'll_last_refresh',
 'll_stable_id',
 'll_updated_at',
 'll_uuid',
 'location_name',
 'lon',
 'lot',
 'mail_addno',
 'mail_addpre

In [ ]:
data_dict = transform_kepler(df)

1. Go to Interactions
2. Add Field to Plot 3D Dimension
3. Go to Layers
4. Click Drop Down on Geojson
5. Turn Off Stroke Color / Width
6. Turn On Height, Click 3 Dots, Select Field


In [ ]:
# create Kepler map
"""
[OPTIONAL] - 3D Steps

1. Go to Interactions
2. Add Field to Plot 3D Dimension
3. Go to Layers
4. Click Drop Down on Geojson
5. Turn Off Stroke Color / Width
6. Turn On Height, Click 3 Dots, Select Field
"""
map = KeplerGl(height=600, data={'Reverse Geocoding Search': data_dict})
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Reverse Geocoding Search': {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geo…

In [ ]:
map.save_to_html(file_name='reverse_geocoding_map.html')

Map saved to reverse_geocoding_map.html!


In [ ]:
download_csv(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color="purple">Use Case #2: Address Search</font>

In [ ]:
# end point
url = "https://app.regrid.com/api/v1/search.json"

# parameters
querystring = {
    "query":"1410 E 31st Ave, Tampa, FL 33603",
    "token":regrid_api_key
}

response = requests.request("GET", url, params=querystring)

print(response.text)

{"results":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-82.4435474018788,27.9787408203148],[-82.4435493712642,27.9790081647977],[-82.4435710590009,27.9790082890018],[-82.4436996362336,27.9790090274115],[-82.443697666478,27.9787416829343],[-82.4435474018788,27.9787408203148]]]},"properties":{"headline":"1410 E 31st Ave","path":"/us/fl/hillsborough/tampa/453886","fields":{"ogc_fid":453886,"geoid":"12057","parcelnumb":"173376.0000","parcelnumb_no_formatting":"1733760000","alt_parcelnumb1":"1733760000","alt_parcelnumb2":"A-07-29-19-4MT-000000-00005.0","usecode":"0100","usedesc":"SINGLE FAMILY","structno":1,"yearbuilt":1926,"numstories":1.0,"numunits":1,"parvaltype":"MARKET","improvval":178541.0,"landval":49994.0,"parval":228535.0,"saleprice":175000.0,"saledate":"2020-08-21","owner":"CYNTHIA AND JERRY OPALKA","mailadd":"1410 E 31ST AVE","mail_city":"TAMPA","mail_state2":"FL","mail_zip":"33603-5109","address":"1410 E 31ST AVE","ll_address_count":1,"ll_bldg_footprint_sqft

In [ ]:
df = pd.json_normalize(response.json(), record_path=['results'])
print('Num of rows:', len(df))
print('Num of columns:', len(df.columns))
df.head()

Num of rows: 10
Num of columns: 251


,type,id,geometry.type,geometry.coordinates,properties.headline,properties.path,properties.fields.ogc_fid,properties.fields.geoid,properties.fields.parcelnumb,properties.fields.parcelnumb_no_formatting,properties.fields.alt_parcelnumb1,properties.fields.alt_parcelnumb2,properties.fields.usecode,properties.fields.usedesc,properties.fields.structno,properties.fields.yearbuilt,properties.fields.numstories,properties.fields.numunits,properties.fields.parvaltype,properties.fields.improvval,properties.fields.landval,properties.fields.parval,properties.fields.saleprice,properties.fields.saledate,properties.fields.owner,properties.fields.mailadd,properties.fields.mail_city,properties.fields.mail_state2,properties.fields.mail_zip,properties.fields.address,properties.fields.ll_address_count,properties.fields.ll_bldg_footprint_sqft,properties.fields.ll_bldg_count,properties.fields.dpv_status,properties.fields.dpv_codes,properties.fields.dpv_notes,properties.fields.dpv_type,properties.fields.cass_errorno,properties.fields.rdi,properties.fields.usps_vacancy,properties.fields.usps_vacancy_date,properties.fields.lbcs_activity,properties.fields.lbcs_activity_desc,properties.fields.lbcs_function,properties.fields.lbcs_function_desc,properties.fields.lbcs_structure,properties.fields.lbcs_structure_desc,properties.fields.lbcs_site,properties.fields.lbcs_site_desc,properties.fields.saddno,properties.fields.saddpref,properties.fields.saddstr,properties.fields.saddsttyp,properties.fields.scity,properties.fields.original_address,properties.fields.city,properties.fields.county,properties.fields.state2,properties.fields.szip,properties.fields.address_source,properties.fields.legaldesc,properties.fields.lat,properties.fields.lon,properties.fields.qoz,properties.fields.qoz_tract,properties.fields.census_tract,properties.fields.census_block,properties.fields.census_blockgroup,properties.fields.ll_last_refresh,properties.fields.recrdareano,properties.fields.gisacre,properties.fields.ll_gisacre,properties.fields.ll_gissqft,properties.fields.plss_township,properties.fields.plss_section,properties.fields.plss_range,properties.fields.path,properties.fields.ll_stable_id,properties.fields.ll_uuid,properties.fields.ll_updated_at,properties.fields.asd_val,properties.fields.sub,properties.fields.strap,properties.fields.taxdist,properties.fields.exf,properties.fields.eff,properties.fields.tax_val,properties.fields.tbeds,properties.fields.tbaths,properties.fields.muni,properties.fields.tif,properties.fields.vi,properties.fields.nbhc,properties.fields.base,properties.fields.just,properties.fields.folio,properties.fields.pin,properties.field_labels.ogc_fid,properties.field_labels.geoid,properties.field_labels.sourceagent,properties.field_labels.parcelnumb,properties.field_labels.parcelnumb_no_formatting,properties.field_labels.alt_parcelnumb1,properties.field_labels.alt_parcelnumb2,properties.field_labels.alt_parcelnumb3,properties.field_labels.usecode,properties.field_labels.usedesc,properties.field_labels.zoning,properties.field_labels.zoning_description,properties.field_labels.struct,properties.field_labels.multistruct,properties.field_labels.structno,properties.field_labels.yearbuilt,properties.field_labels.numstories,properties.field_labels.numunits,properties.field_labels.structstyle,properties.field_labels.parvaltype,properties.field_labels.improvval,properties.field_labels.landval,properties.field_labels.parval,properties.field_labels.agval,properties.field_labels.saleprice,properties.field_labels.saledate,properties.field_labels.taxamt,properties.field_labels.taxyear,properties.field_labels.owntype,properties.field_labels.owner,properties.field_labels.ownfrst,properties.field_labels.ownlast,properties.field_labels.owner2,properties.field_labels.owner3,properties.field_labels.owner4,properties.field_labels.subsurfown,properties.field_labels.subowntype,properties.field_labels.mailadd,properties.field_labels.mail_address2,properties.field_labels.careof,propertie

In [ ]:
df[['properties.headline', 'properties.fields.parcelnumb', 'properties.fields.usedesc', 
    'properties.fields.numstories',	'properties.fields.numunits', 'properties.fields.improvval',	
    'properties.fields.landval',	'properties.fields.parval', 'properties.fields.saleprice',
    'properties.fields.saledate',	'properties.fields.owner',
    'properties.fields.census_tract',	'properties.fields.census_block',	'properties.fields.census_blockgroup'
    ]].head()

,properties.headline,properties.fields.parcelnumb,properties.fields.usedesc,properties.fields.numstories,properties.fields.numunits,properties.fields.improvval,properties.fields.landval,properties.fields.parval,properties.fields.saleprice,properties.fields.saledate,properties.fields.owner,properties.fields.census_tract,properties.fields.census_block,properties.fields.census_blockgroup
0,1410 E 31st Ave,173376.0000,SINGLE FAMILY,1.0,1,178541.0,49994.0,228535.0,175000.0,2020-08-21,CYNTHIA AND JERRY OPALKA,12057003100,120570031002004,120570031002
1,1410 E Louise Ave,172920.0000,SINGLE FAMILY,1.0,1,70244.0,100464.0,178401.0,0.0,NaN,RUBEN AND CELIA ANGELICA OCAMPO/TRUSTEES,12057003100,120570031002003,120570031002
2,1410 E Louisiana Ave,171873.0000,SINGLE FAMILY,1.0,1,176706.0,30696.0,208024.0,259900.0,2021-11-29,KARA EVERLYN MCGOVERN,12057002100,120570021002016,120570021002
3,1410 E Lake Ave,172522.0000,SINGLE FAMILY,1.0,1,126591.0,74812.0,201403.0,175000.0,2020-10-28,ERIC SCOLA,12057003100,120570031002004,120570031002
4,1410 E Curtis St Unit 1,171921.0000,SINGLE FAMILY,2.0,1,370694.0,41188.0,411882.0,397000.0,2021-08-26,ARHTUR AND ZBIGNIEW KAZIMIEREZ HOROCKI,12057002100,120570021002027,120570021002


In [ ]:
df['properties.fields.owner']

0                    CYNTHIA AND JERRY OPALKA
1    RUBEN AND CELIA ANGELICA OCAMPO/TRUSTEES
2                       KARA EVERLYN MCGOVERN
3                                  ERIC SCOLA
4      ARHTUR AND ZBIGNIEW KAZIMIEREZ HOROCKI
5                    AMMA AND KENNETH A TUNER
6                      ALVIN AND RENEE HOLDER
7                           THADDAEUS JACKSON
8                                    ALICE YU
9      KERNET L DALPHINS AND THADDEUS FRANCIS
Name: properties.fields.owner, dtype: object

In [ ]:
download_csv(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color="purple">Use Case #3: Assessor Parcel Number Search</font> <i>(Single Record)</i>

In [ ]:
# end point
url = "https://app.regrid.com/api/v1/search.json"

# parameters
querystring = {
    "parcelnumb":"A-07-29-19-4MT-000000-00005.0",
    "context":"us/fl/hillsborough/tampa",
    "token":regrid_api_key
}

response = requests.request("GET", url, params=querystring)

print(response.text)

{"results":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-82.4435474018788,27.9787408203148],[-82.4435493712642,27.9790081647977],[-82.4435710590009,27.9790082890018],[-82.4436996362336,27.9790090274115],[-82.443697666478,27.9787416829343],[-82.4435474018788,27.9787408203148]]]},"properties":{"headline":"1410 E 31st Ave","path":"/us/fl/hillsborough/tampa/453886","fields":{"ogc_fid":453886,"geoid":"12057","parcelnumb":"173376.0000","parcelnumb_no_formatting":"1733760000","alt_parcelnumb1":"1733760000","alt_parcelnumb2":"A-07-29-19-4MT-000000-00005.0","usecode":"0100","usedesc":"SINGLE FAMILY","structno":1,"yearbuilt":1926,"numstories":1.0,"numunits":1,"parvaltype":"MARKET","improvval":178541.0,"landval":49994.0,"parval":228535.0,"saleprice":175000.0,"saledate":"2020-08-21","owner":"CYNTHIA AND JERRY OPALKA","mailadd":"1410 E 31ST AVE","mail_city":"TAMPA","mail_state2":"FL","mail_zip":"33603-5109","address":"1410 E 31ST AVE","ll_address_count":1,"ll_bldg_footprint_sqft

In [ ]:
df = pd.json_normalize(response.json(), record_path=['results'])
print('Num of rows:', len(df))
print('Num of columns:', len(df.columns))
df.head()

Num of rows: 1
Num of columns: 250


,type,id,geometry.type,geometry.coordinates,properties.headline,properties.path,properties.fields.ogc_fid,properties.fields.geoid,properties.fields.parcelnumb,properties.fields.parcelnumb_no_formatting,properties.fields.alt_parcelnumb1,properties.fields.alt_parcelnumb2,properties.fields.usecode,properties.fields.usedesc,properties.fields.structno,properties.fields.yearbuilt,properties.fields.numstories,properties.fields.numunits,properties.fields.parvaltype,properties.fields.improvval,properties.fields.landval,properties.fields.parval,properties.fields.saleprice,properties.fields.saledate,properties.fields.owner,properties.fields.mailadd,properties.fields.mail_city,properties.fields.mail_state2,properties.fields.mail_zip,properties.fields.address,properties.fields.ll_address_count,properties.fields.ll_bldg_footprint_sqft,properties.fields.ll_bldg_count,properties.fields.dpv_status,properties.fields.dpv_codes,properties.fields.dpv_notes,properties.fields.dpv_type,properties.fields.cass_errorno,properties.fields.rdi,properties.fields.usps_vacancy,properties.fields.usps_vacancy_date,properties.fields.lbcs_activity,properties.fields.lbcs_activity_desc,properties.fields.lbcs_function,properties.fields.lbcs_function_desc,properties.fields.lbcs_structure,properties.fields.lbcs_structure_desc,properties.fields.lbcs_site,properties.fields.lbcs_site_desc,properties.fields.saddno,properties.fields.saddpref,properties.fields.saddstr,properties.fields.saddsttyp,properties.fields.scity,properties.fields.original_address,properties.fields.city,properties.fields.county,properties.fields.state2,properties.fields.szip,properties.fields.address_source,properties.fields.legaldesc,properties.fields.lat,properties.fields.lon,properties.fields.qoz,properties.fields.qoz_tract,properties.fields.census_tract,properties.fields.census_block,properties.fields.census_blockgroup,properties.fields.ll_last_refresh,properties.fields.recrdareano,properties.fields.gisacre,properties.fields.ll_gisacre,properties.fields.ll_gissqft,properties.fields.plss_township,properties.fields.plss_section,properties.fields.plss_range,properties.fields.path,properties.fields.ll_stable_id,properties.fields.ll_uuid,properties.fields.ll_updated_at,properties.fields.asd_val,properties.fields.sub,properties.fields.strap,properties.fields.taxdist,properties.fields.exf,properties.fields.eff,properties.fields.tax_val,properties.fields.tbeds,properties.fields.tbaths,properties.fields.muni,properties.fields.tif,properties.fields.vi,properties.fields.nbhc,properties.fields.base,properties.fields.just,properties.fields.folio,properties.fields.pin,properties.field_labels.ogc_fid,properties.field_labels.geoid,properties.field_labels.sourceagent,properties.field_labels.parcelnumb,properties.field_labels.parcelnumb_no_formatting,properties.field_labels.alt_parcelnumb1,properties.field_labels.alt_parcelnumb2,properties.field_labels.alt_parcelnumb3,properties.field_labels.usecode,properties.field_labels.usedesc,properties.field_labels.zoning,properties.field_labels.zoning_description,properties.field_labels.struct,properties.field_labels.multistruct,properties.field_labels.structno,properties.field_labels.yearbuilt,properties.field_labels.numstories,properties.field_labels.numunits,properties.field_labels.structstyle,properties.field_labels.parvaltype,properties.field_labels.improvval,properties.field_labels.landval,properties.field_labels.parval,properties.field_labels.agval,properties.field_labels.saleprice,properties.field_labels.saledate,properties.field_labels.taxamt,properties.field_labels.taxyear,properties.field_labels.owntype,properties.field_labels.owner,properties.field_labels.ownfrst,properties.field_labels.ownlast,properties.field_labels.owner2,properties.field_labels.owner3,properties.field_labels.owner4,properties.field_labels.subsurfown,properties.field_labels.subowntype,properties.field_labels.mailadd,properties.field_labels.mail_address2,properties.field_labels.careof,propertie

In [ ]:
download_csv(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color="purple">Use Case #4: Polygon Search</font>

In [ ]:
# coming soon!

In [ ]:
download_csv(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color="purple">Use Case #5: Owner Name Search</font>

In [ ]:
# end point
url = "https://app.regrid.com/api/v1/search.json"

# parameters
querystring = {
    "owner":"NATIONAL ASSETS & COMPANY LLC",
    "context":"us/fl/hillsborough/tampa",
    "token":regrid_api_key
}

response = requests.request("GET", url, params=querystring)

print(response.text)

{"results":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-82.5354470032002,28.1003342830101],[-82.535684045522,28.1005476470073],[-82.535838632467,28.1004096567493],[-82.5356041951636,28.1001960626606],[-82.5354470032002,28.1003342830101]]]},"properties":{"headline":"4927 Pennsbury Dr","path":"/us/fl/hillsborough/keystone-citrus-park/67154","fields":{"ogc_fid":67154,"geoid":"12057","parcelnumb":"16252.2018","parcelnumb_no_formatting":"162522018","alt_parcelnumb1":"0162522018","alt_parcelnumb2":"U-30-27-18-0PZ-000014-00029.0","usecode":"0100","usedesc":"SINGLE FAMILY","structno":1,"yearbuilt":1984,"numstories":1.0,"numunits":1,"parvaltype":"MARKET","improvval":179753.0,"landval":72993.0,"parval":254020.0,"saleprice":129000.0,"saledate":"2009-12-02","owner":"NATIONAL ASSETS & COMPANY LLC","mailadd":"3959 VAN DYKE RD UNIT 193","mail_city":"LUTZ","mail_state2":"FL","mail_zip":"33558-8029","address":"4927 PENNSBURY DR","ll_address_count":1,"ll_bldg_footprint_sqft":2676,"l

In [ ]:
df = pd.json_normalize(response.json(), record_path=['results'])
print('Num of rows:', len(df))
print('Num of columns:', len(df.columns))
df.head()

Num of rows: 10
Num of columns: 376


,type,id,geometry.type,geometry.coordinates,properties.headline,properties.path,properties.fields.ogc_fid,properties.fields.geoid,properties.fields.parcelnumb,properties.fields.parcelnumb_no_formatting,properties.fields.alt_parcelnumb1,properties.fields.alt_parcelnumb2,properties.fields.usecode,properties.fields.usedesc,properties.fields.structno,properties.fields.yearbuilt,properties.fields.numstories,properties.fields.numunits,properties.fields.parvaltype,properties.fields.improvval,properties.fields.landval,properties.fields.parval,properties.fields.saleprice,properties.fields.saledate,properties.fields.owner,properties.fields.mailadd,properties.fields.mail_city,properties.fields.mail_state2,properties.fields.mail_zip,properties.fields.address,properties.fields.ll_address_count,properties.fields.ll_bldg_footprint_sqft,properties.fields.ll_bldg_count,properties.fields.dpv_status,properties.fields.dpv_codes,properties.fields.dpv_notes,properties.fields.dpv_type,properties.fields.cass_errorno,properties.fields.rdi,properties.fields.usps_vacancy,properties.fields.usps_vacancy_date,properties.fields.lbcs_activity,properties.fields.lbcs_activity_desc,properties.fields.lbcs_function,properties.fields.lbcs_function_desc,properties.fields.lbcs_structure,properties.fields.lbcs_structure_desc,properties.fields.lbcs_site,properties.fields.lbcs_site_desc,properties.fields.saddno,properties.fields.saddstr,properties.fields.saddsttyp,properties.fields.scity,properties.fields.original_address,properties.fields.city,properties.fields.county,properties.fields.state2,properties.fields.szip,properties.fields.address_source,properties.fields.legaldesc,properties.fields.lat,properties.fields.lon,properties.fields.qoz,properties.fields.census_tract,properties.fields.census_block,properties.fields.census_blockgroup,properties.fields.ll_last_refresh,properties.fields.recrdareano,properties.fields.gisacre,properties.fields.ll_gisacre,properties.fields.ll_gissqft,properties.fields.plss_township,properties.fields.plss_section,properties.fields.plss_range,properties.fields.path,properties.fields.ll_stable_id,properties.fields.ll_uuid,properties.fields.ll_updated_at,properties.fields.asd_val,properties.fields.sub,properties.fields.strap,properties.fields.taxdist,properties.fields.exf,properties.fields.eff,properties.fields.tax_val,properties.fields.tbeds,properties.fields.tbaths,properties.fields.muni,properties.fields.sd1,properties.fields.vi,properties.fields.nbhc,properties.fields.base,properties.fields.just,properties.fields.folio,properties.fields.pin,properties.field_labels.ogc_fid,properties.field_labels.geoid,properties.field_labels.sourceagent,properties.field_labels.parcelnumb,properties.field_labels.parcelnumb_no_formatting,properties.field_labels.alt_parcelnumb1,properties.field_labels.alt_parcelnumb2,properties.field_labels.alt_parcelnumb3,properties.field_labels.usecode,properties.field_labels.usedesc,properties.field_labels.zoning,properties.field_labels.zoning_description,properties.field_labels.struct,properties.field_labels.multistruct,properties.field_labels.structno,properties.field_labels.yearbuilt,properties.field_labels.numstories,properties.field_labels.numunits,properties.field_labels.structstyle,properties.field_labels.parvaltype,properties.field_labels.improvval,properties.field_labels.landval,properties.field_labels.parval,properties.field_labels.agval,properties.field_labels.saleprice,properties.field_labels.saledate,properties.field_labels.taxamt,properties.field_labels.taxyear,properties.field_labels.owntype,properties.field_labels.owner,properties.field_labels.ownfrst,properties.field_labels.ownlast,properties.field_labels.owner2,properties.field_labels.owner3,properties.field_labels.owner4,properties.field_labels.subsurfown,properties.field_labels.subowntype,properties.field_labels.mailadd,properties.field_labels.mail_address2,properties.field_labels.careof,properties.field_labels.mail_addno,properties.field_labels.mail_

In [ ]:
# types of properties owned
df.groupby(['properties.fields.usedesc'])['id'].count()

properties.fields.usedesc
LIGHT INDUS.      1
SINGLE FAMILY     7
VACANT ACREAGE    1
Name: id, dtype: int64

In [ ]:
# copy original df and add features
df_plot = df.copy()
# convert lat and lon fields to numeric
df_plot["properties.fields.lat"] = df_plot["properties.fields.lat"].astype(float)
df_plot["properties.fields.lon"] = df_plot["properties.fields.lon"].astype(float)
# create UNKNOWN field for null descriptions 
df_plot["prop_type"] = df_plot.apply(
    lambda x: 'UNKNOWN' if pd.isnull(x["properties.fields.usedesc"]) else x["properties.fields.usedesc"], axis=1)

# plotly maps - https://plotly.com/python/scattermapbox/
px.set_mapbox_access_token(mapbox_api_key)
fig = px.scatter_mapbox(
    df_plot, 
    hover_name="properties.headline",
    # coordinates
    lat="properties.fields.lat", 
    lon="properties.fields.lon",  
    # group by   
    color="prop_type", 
    # size of markers
    size="properties.fields.ll_gissqft",
    color_continuous_scale=px.colors.cyclical.IceFire, 
    size_max=15, 
    zoom=9
)
fig.show()

### <font color="purple">Use Case #6: Query by additional parcel data fields (WIP)</font>

In [ ]:
import requests

response = requests.get('https://app.regrid.com/api/v1/query?fields[zoning][ilike]=RS-1&token=STKA2Fd1s-VnjmnsWFzgK-Y_Wo-V_P82Q_oxmE2EfhcCkve1byWP332Djx7JxwA_')
print(response.text)

{"results":[{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-119.835577992809,36.8390633244439],[-119.835577993102,36.8390633514775],[-119.835888934048,36.8390618732532],[-119.836432631653,36.8390592857532],[-119.836433295386,36.8386883308015],[-119.835578659821,36.8386923965388],[-119.835577992809,36.8390633244439]]]},"properties":{"headline":"7089 N Van Ness Blvd","path":"/us/ca/fresno/fresno/826638","fields":{"ogc_fid":826638,"geoid":"06019","parcelnumb":"50026122S","parcelnumb_no_formatting":"50026122S","usecode":"S01","zoning":"RS-1","improvval":1060000.0,"landval":530000.0,"parval":1590000.0,"homestead_exemption":"0","owner":"LION JEFFREY A","mailadd":"7089 N VAN NESS","mail_city":"FRESNO","mail_state2":"CA","mail_zip":"93711","address":"7089 N VAN NESS BLVD","saddno":"7089","saddpref":"N","saddstr":"VAN NESS","saddsttyp":"BLVD","scity":"FRESNO","original_address":"7089 N VAN NESS BLVD; 7089; VAN NESS","city":"fresno","county":"fresno","state2":"ca","szip":"93711-

In [ ]:
df = pd.json_normalize(response.json(), record_path=['results'])
print('Num of rows:', len(df))
print('Num of columns:', len(df.columns))
df.tail()

Num of rows: 20
Num of columns: 79


,type,id,geometry.type,geometry.coordinates,properties.headline,properties.path,properties.fields.ogc_fid,properties.fields.geoid,properties.fields.parcelnumb,properties.fields.parcelnumb_no_formatting,properties.fields.usecode,properties.fields.zoning,properties.fields.improvval,properties.fields.landval,properties.fields.parval,properties.fields.homestead_exemption,properties.fields.owner,properties.fields.mailadd,properties.fields.mail_city,properties.fields.mail_state2,properties.fields.mail_zip,properties.fields.address,properties.fields.saddno,properties.fields.saddpref,properties.fields.saddstr,properties.fields.saddsttyp,properties.fields.scity,properties.fields.original_address,properties.fields.city,properties.fields.county,properties.fields.state2,properties.fields.szip,properties.fields.ll_address_count,properties.fields.address_source,properties.fields.legaldesc,properties.fields.lat,properties.fields.lon,properties.fields.qoz,properties.fields.census_tract,properties.fields.census_block,properties.fields.census_blockgroup,properties.fields.sqft,properties.fields.ll_gisacre,properties.fields.ll_gissqft,properties.fields.ll_bldg_footprint_sqft,properties.fields.ll_bldg_count,properties.fields.plss_township,properties.fields.plss_section,properties.fields.plss_range,properties.fields.path,properties.fields.ll_uuid,properties.fields.ll_updated_at,properties.fields.dpv_status,properties.fields.dpv_codes,properties.fields.dpv_notes,properties.fields.dpv_type,properties.fields.cass_errorno,properties.fields.rdi,properties.fields.usps_vacancy,properties.fields.usps_vacancy_date,properties.fields.lbcs_activity,properties.fields.lbcs_activity_desc,properties.fields.lbcs_function,properties.fields.lbcs_function_desc,properties.fields.lbcs_structure,properties.fields.lbcs_structure_desc,properties.fields.lbcs_site,properties.fields.lbcs_site_desc,properties.fields.pers_prop_val,properties.fields.tax_area_code,properties.fields.use_high_best,properties.fields.use_secondary,properties.fields.instrument_number,properties.ll_uuid,properties.fields.ll_stable_id,properties.fields.careof,properties.fields.qoz_tract,properties.fields.lbcs_ownership,properties.fields.lbcs_ownership_desc
15,Feature,7882530,Polygon,"[[[-119.829904162084, 36.8190193605792], [-119...",41503437T,/us/ca/fresno/fresno/379248,379248,06019,41503437T,41503437T,NaN,RS-1,NaN,NaN,NaN,NaN,FRESNO CITY OF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,NaN,fresno,fresno,ca,93711,0,"county,census_places",NaN,36.818862,-119.829856,No,06019004303,060190043031000,060190043031,NaN,0.07046,3069,NaN,0,013S,Section 07,020E,/us/ca/fresno/fresno/379248,0851c81c-0d48-4e57-b91c-abc6741c3f88,2022-05-19 10:27:54 -0400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1100.0,Household,1100.0,Private household,1000.0,Residential buildings,6000.0,Developed site with buildings,NaN,005784,NaN,NaN,NaN,0851c81c-0d48-4e57-b91c-abc6741c3f88,preserved,NaN,NaN,4110.0,"City, Village, Township, etc."
16,Feature,7887924,Polygon,"[[[-119.834189821938, 36.8094689148159], [-119...",41524011,/us/ca/fresno/fresno/337948,337948,06019,41524011,41524011,S01,RS-1,127232.0,41658.0,168890.0,7000,CHERRY ROBERT T & TAY N TRUSTEES,5001 N VAN NESS,FRESNO,CA,93711,NaN,NaN,NaN,NaN,NaN,Fresno,NaN,fresno,fresno,ca,93711,0,"county,census_places",CONDOMINIUM UNIT B3 TRACT 2267 CASITAS VAN NESS,36.809555,-119.833995,No,06019004303,060190043032017,060190043032,0.09,0.14677,6394,1241.0,1,013S,Section 07,020E,/us/ca/fresno/fresno/337948,56f54140-7cc5-4afb-9ab3-5e931c127e9c,2022-05-19 10:27:54 -0400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1100.0,Household,1100.0,Private household,1000.0,Residential buildings,6000.0,Developed site with buildings,0.0,005001,P,C48,065310,56f54140-7cc5-4afb-9ab3-5e931c127e9c,preserved,NaN,NaN,NaN,NaN
17,Feature,7939106,Polygon,"[[[-119.835149038977, 36.8492641443141], [-119...",40503017,/us/ca/fresno/fresno/345599,345599,06019,40503017,40503017,000,RS-1,0.0,2442.0,2442.0,0,CLOUD STEPHEN R SR TRUSTEE,P O BOX 6078,FRESNO,

In [ ]:
df.groupby(['properties.fields.zoning'])['id']\
  .count().reset_index().rename(columns={'id': 'count'})\
  .sort_values(by=['count'], ascending=False)

,properties.fields.zoning,count
0,RS-1,20


In [ ]:
# end point
url = "https://app.regrid.com/api/v1/query"

# parameters
querystring = {
  "fields[geoid][eq]":"26163", # fips code [Wayne County Detroit MI], "06019" [Fresno County]
  "limit":"5",
  "token":regrid_api_key,
  "fields[zoning][ilike]": "RS-1"
}

response = requests.request("GET", url, params=querystring)

print(response.text)

{"results":[]}


In [ ]:
df = pd.json_normalize(response.json(), record_path=['results'])
print('Num of rows:', len(df))
print('Num of columns:', len(df.columns))
df.tail()

Num of rows: 0
Num of columns: 0


""


In [ ]:
df.groupby(['properties.fields.zoning'])['id']\
  .count().reset_index().rename(columns={'id': 'count'})\
  .sort_values(by=['count'], ascending=False)

KeyError: ignored

In [ ]:
df.groupby(['properties.fields.usedesc'])['id']\
  .count().reset_index().rename(columns={'id': 'count'})\
  .sort_values(by=['count'], ascending=False)

In [ ]:
# end point
url = "https://app.regrid.com/api/v1/search.json"

# parameters
querystring = {
    "owner":"NATIONAL ASSETS & COMPANY LLC",
    "context":"us/fl/hillsborough/tampa",
    "token":regrid_api_key
}

response = requests.request("GET", url, params=querystring)

print(response.text)

In [ ]:
# end point
url = "https://app.regrid.com/api/v1/search.json"
# url = """https://app.regrid.com/api/v1/query?fields[zoning][ilike]=RS-1&fields[zoning][ne]=RS-1&fields[zoning][between]=["RS-1", "RS-14"]&fields[zoning][nin]=["RS-1", "RS-9", "MHPD", "RD-D", "AG-2"]&fields[zoning][order]=DESC"""
# url = """https://app.regrid.com/api/v1/query?fields[zoning][ilike]=RS-1"""
# # parameters
# querystring = {
#     "owner":"NATIONAL ASSETS & COMPANY LLC",
#     "context":"us/fl/hillsborough/tampa",
#     "token":regrid_api_key
# }

# response = requests.request("GET", url, params=querystring)


querystring = {
    # "owner":"NATIONAL ASSETS & COMPANY LLC",
    "limit":15,
    "context":"/us/mi/wayne/detroit",#"us/fl/hillsborough/tampa",
    "token":regrid_api_key
}

response = requests.request("GET", url, params=querystring)

print(response.text)

In [ ]:
regrid_api_key

In [ ]:
import requests

response = requests.get("""https://app.regrid.com/api/v1/query?fields[zoning][ilike]=RS-1&fields[zoning][ne]=RS-1&fields[zoning][between]=["RS-1", "RS-14"]&fields[zoning][nin]=["RS-1", "RS-9", "MHPD", "RD-D", "AG-2"]&fields[zoning][order]=DESC&token=STKA2Fd1s-VnjmnsWFzgK-Y_Wo-V_P82Q_oxmE2EfhcCkve1byWP332Djx7JxwA_""")
print(response.status_code)
print(response.text)

## <font color="blue">Output</font>

In [ ]:
# download_csv(df)

# End Notebook